# Simulated Annealing

Gradient descent 방법을 이용하면 local minimum/maximum을 찾을 수 있으나 퍼텐셜이 복잡한 형태라면 global minimum/maximum을 찾고자 한다면 거의 항상 실패할 것이다. 이를 극복하기 위한 방법 중 하나가 simulated annealing(모의담금질)이다.

## Traveling Salesman 